In [3]:
%pip install -q timm


Note: you may need to restart the kernel to use updated packages.


In [1]:
# main.py
import argparse
# === One-cell runner (Known-T) ===
import sys, numpy as np
sys.path.append(".")  # ensure current folder is importable

from core import run_known_T, parse_T_str

NPZ = r"datasets/FashionMNIST0.3.npz"   # <- change to your file
# Replace with your actual 3x3 transition matrix, row-major:
T_str = "0.9,0.05,0.05,0.05,0.9,0.05,0.05,0.05,0.9"

mean, std = run_known_T(
    npz_path=NPZ,
    T=parse_T_str(T_str),
    trials=2,          # start small to test
    grayscale=True,    # important: tells the loader to expect single-channel
    epochs=1,          # quick smoke test
)
print(mean, std)


C:\Users\William\desktop\comp5328-Assign2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.6925 0.013499999999999956


In [2]:
print(mean, std)

0.6925 0.013499999999999956


In [2]:
from core import run_known_T, parse_T_str
NPZ = r"datasets/FashionMNIST0.3.npz"
T_str = "0.7,0.3,0, 0,0.7,0.3, 0.3,0,0.7"  # replace with your exact matrix if different

mean, std = run_known_T(
    npz_path=NPZ,
    T=parse_T_str(T_str),
    trials=5,          # use 10 for final
    grayscale=True,    # FashionMNIST
    model_name="vit_tiny_patch16_224",
    epochs=15,
    batch_size=128,
    loss_name="forward"  # or "gce"/"bootstrap" for your 2nd classifier
)
print(f"FashionMNIST0.3 ViT (forward-T): mean={mean:.4f} std={std:.4f}")


FashionMNIST0.3 ViT (forward-T): mean=0.9653 std=0.0063


In [1]:
from core import run_known_T, parse_T_str
NPZ = r"datasets/FashionMNIST0.6.npz"
T_str = "0.4,0.3,0.3, 0.3,0.4,0.3, 0.3,0.3,0.4"

mean, std = run_known_T(
    npz_path=NPZ, T=parse_T_str(T_str),
    trials=5, grayscale=True, epochs=15, batch_size=128,
    loss_name="forward"
)
print(f"FashionMNIST0.6 ViT (forward-T): mean={mean:.4f} std={std:.4f}")


C:\Users\William\desktop\comp5328-Assign2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FashionMNIST0.6 ViT (forward-T): mean=0.4227 std=0.1001


In [6]:
from core import run_unknown_T
NPZ = r"datasets/CIFAR.npz"

mean, std = run_unknown_T(
    npz_path=NPZ,
    trials=5,            # use 10 for final
    grayscale=False,     # CIFAR is RGB
    warmup_epochs=5,     # warm-up before estimating T̂
    epochs=15,
    batch_size=128
)
print(f"CIFAR ViT (estimate T̂): mean={mean:.4f} std={std:.4f}")


CIFAR ViT (estimate T̂): mean=0.4073 std=0.0252


In [8]:
import importlib, core
importlib.reload(core)
import numpy as np
from core import run_known_T, parse_T_str

NPZ = r"datasets/FashionMNIST0.6.npz"
T = parse_T_str("0.4,0.3,0.3, 0.3,0.4,0.3, 0.3,0.3,0.4")

m1,s1 = run_known_T(NPZ, T,   trials=3, grayscale=True, epochs=5)
m2,s2 = run_known_T(NPZ, T.T, trials=3, grayscale=True, epochs=5)
print("0.6 with T   ->", m1, s1)
print("0.6 with T.T ->", m2, s2)


0.6 with T   -> 0.3828888888888889 0.06642028546300939
0.6 with T.T -> 0.4802222222222223 0.1042661598926777


In [9]:
import numpy as np, itertools, torch, torch.nn as nn
from core import build_loaders_from_npz, make_vit, estimate_T_confident, run_known_T

NPZ = r"datasets/FashionMNIST0.6.npz"
T_provided = np.array([[0.4,0.3,0.3],
                       [0.3,0.4,0.3],
                       [0.3,0.3,0.4]], dtype=np.float32)

# --- warm-up to get a decent classifier ---
train_loader, val_loader, _ = build_loaders_from_npz(NPZ, grayscale=True, batch_size=256)
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = make_vit(num_classes=3).to(dev)
opt = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05)
ce  = nn.CrossEntropyLoss()
for _ in range(3):  # short warm-up
    model.train()
    for xb,yb in train_loader:
        xb,yb = xb.to(dev), yb.to(dev)
        opt.zero_grad(); loss = ce(model(xb), yb); loss.backward(); opt.step()

# --- estimate T_hat (rows=clean, cols=noisy) ---
T_hat = estimate_T_confident(model, train_loader, dev, top_k=200, num_classes=3).cpu().numpy()

def permute_T(T, perm):
    """perm is a tuple like (2,0,1) mapping clean/noisy class order.
       Applies P^T T P (reindex rows and columns)."""
    P = np.eye(3, dtype=np.float32)[list(perm)]
    return P.T @ T @ P

best = None
for perm in itertools.permutations([0,1,2]):
    for use_TT in [False, True]:
        T_candidate = T_provided.T if use_TT else T_provided
        T_perm = permute_T(T_candidate, perm)
        l1 = np.abs(T_hat - T_perm).sum()
        tag = f"{'T.T' if use_TT else 'T'} with perm {perm}, L1={l1:.4f}"
        if (best is None) or (l1 < best[0]):
            best = (l1, use_TT, perm, T_perm, tag)
        print(tag)

print("\nBEST:", best[4])

# --- run with the best-aligned matrix (few trials first) ---
T_fixed = best[3]
m,s = run_known_T(NPZ, T_fixed, trials=3, grayscale=True, epochs=8)
print("Fixed 0.6 ->", m, s)


T with perm (0, 1, 2), L1=0.3100
T.T with perm (0, 1, 2), L1=0.3100
T with perm (0, 2, 1), L1=0.3100
T.T with perm (0, 2, 1), L1=0.3100
T with perm (1, 0, 2), L1=0.3100
T.T with perm (1, 0, 2), L1=0.3100
T with perm (1, 2, 0), L1=0.3100
T.T with perm (1, 2, 0), L1=0.3100
T with perm (2, 0, 1), L1=0.3100
T.T with perm (2, 0, 1), L1=0.3100
T with perm (2, 1, 0), L1=0.3100
T.T with perm (2, 1, 0), L1=0.3100

BEST: T with perm (0, 1, 2), L1=0.3100
Fixed 0.6 -> 0.4283333333333334 0.06786642654399784


In [10]:
from core import run_known_T, parse_T_str
NPZ = r"datasets/FashionMNIST0.6.npz"
T = parse_T_str("0.4,0.3,0.3, 0.3,0.4,0.3, 0.3,0.3,0.4")

mean, std = run_known_T(
    npz_path=NPZ,
    T=T.T,                 # <-- transpose
    trials=10,             # final grading runs
    grayscale=True,        # FashionMNIST
    epochs=20,             # give it more runway at 60% noise
    batch_size=128,
)
print(mean, std)


0.4141666666666667 0.10177395977797508
